# Análise Comparativa

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

1. Obtenção dos dados


Nessa fase do processo obtemos o conjunto de dados e dicionário de dados.

In [2]:
df = pd.read_csv('../data/raw/obesity_dataset.csv')
df_dict = pd.read_csv('../data/external/dictionary.csv')

df_dict

,variavel,descricao,tipo,subtipo
0,Age,idade do indivíduo,quantitativa,contínua
1,Gender,gênero do indivíduo,qualitativa,nominal
2,Height,altura do indiíduo,quantitativa,contínua
3,Weight,peso do indivíduo,quantitativa,contínua
4,CALC,frequência do consumo de álcool pelo indivíduo,qualitativa,ordinal
5,FAVC,indica se o indivíduo consome comidas altament...,qualitativa,nominal
6,FCVC,indica o nível de consumo de vegetais nas refe...,quantitativa,discreta
7,NCP,quantas refeições principais o indivíduo faz d...,quantitativa,contínua
8,SCC,indica se o indivíduo monitora as calorias ing...,qualitativa,nominal
9,SMOKE,indica se o indivíduo fuma ou não,qualitativa,nominal


2. Preparação dos dados

Nessa fase realizamos as principais transformações no conjunto de dados bruto já visando a modelagem.

Para as variáveis contínuas, utilizamos a técnica do [OneHotEncoder](http://https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) para fazer o encoding das variáveis categóricas e utilizando o [StandardScaler](http://https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#standardscaler) para normalização das variáveis contínuas.

Não aplicamos nenhum método para tratar dados ausentes, pois o conjunto de dados não possui valores faltantes.

In [3]:
target_column = 'NObeyesdad'
colunas_nominal = (
    df_dict
    .query("subtipo == 'nominal'") # Filtrando colunas nominais e tirando a variavel alvo.
    .variavel
    .to_list()
)
colunas_continua = (
    df_dict
    .query("subtipo == 'contínua'") # Filtrando colunas contínuas.
    .variavel
    .to_list()
)
colunas_ordinal = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_column") # Filtrando colunas ordinais e tirando a variavel alvo.
    .variavel
    .to_list()
)
colunas_discreta = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1) # Dropando a variável alvo para ficarmos somente com as variáveis independentes.
y = df[target_column]

In [4]:
preprocessador_nominal = Pipeline([ 
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), 
])
preprocessador_continua = Pipeline([ 
    ('normalization', StandardScaler())
])
preprocessador_ordinal = Pipeline([
    ('encoding', OrdinalEncoder()),
    ('normalization', StandardScaler())
    
])
preprocessador_discreta = Pipeline([
    ('normalization', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('nominal', preprocessador_nominal, colunas_nominal),
    ('continuous', preprocessador_continua, colunas_continua),
    ('ordinal', preprocessador_ordinal, colunas_ordinal),
    ('discreta', preprocessador_discreta, colunas_discreta)
])


3. Seleção de modelos

Iremos análisar quatro modelos, que serão testados utilizando o KFold como método de validação, a saber:

- Logistic Regression
- K-Nearest-Neighbors
- Support Vector Machine
- Decision Tree

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

Acurácia (accuracy): proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
Precisão (precision): proporção entre dados corretamente previstos como positivos e o total de observações positivas.
Recall: proporção entre dados corretamente previstos como positivos com o total de observações.

In [5]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'accuracy' # Métrica utilizada pelo GridSearchCV para avaliar e comparar o desempenho do modelo durante o processo de ajuste de hiperparâmetros.
metrics = ['accuracy', 'precision_macro', 'recall_macro'] # A média macro considera cada classe individualmente e calcula a média das métricas sem ponderar pelo número de instâncias em cada classe.

# model settings
max_iter = 1000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance'], 'metrics':['minkowski', 'euclidean', 'manhattan'], 'p':[1,2]})
]